In [6]:
import pysam
import numpy as np
from Bio import SeqIO
import numpy as np
import scipy.stats as ss
import pandas as pd

In [4]:
# Метод вычислений:
# 0.1 ) скорректировал сырые риды с помощью BayesHammer
# 0.2 ) выровнил исходные(с ошибками) риды на геном, а затем исправленные
# 1) последовательно просматриваю пары вида error_read, corrected_read (ореинтируясь на read_id) на некоторый участок генома
#    и вычисляю значения из таблицы для позиции i в ридах следующим образом:
#        undetected_error - считаю, если значение base в error_read, corrected_read не совпадает с геномом и при этом равны между собой
#        incorectly_removed_base - считаю, если в error_read ошибки нет, а в corrected_read стоит буква 'N'
#        falsely_corrected_error - считаю, если в error_read ошибки нет, а в corrected_read стоит буква отличная от буквы в геноме (не 'N')
#        detected_and_corrected_error - считаю, если в error_read была ошибка, а в corrected_error её нет
#        detected_and_removed_error - считаю, если в error_read была ошибка, а в corrected_error и в геноме стоит буква 'N'
#        correctly_unmodified_base - считаю, если в error_read и в corrected_read не было ошибки

In [5]:
!bwa index data/MG1655-K12.first400K.fasta

[bwa_index] Pack FASTA... 0.01 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.18 seconds elapse.
[bwa_index] Update BWT... 0.01 sec
[bwa_index] Pack forward-only FASTA... 0.01 sec
[bwa_index] Construct SA from BWT and Occ... 0.07 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index data/MG1655-K12.first400K.fasta
[main] Real time: 0.660 sec; CPU: 0.290 sec


In [ ]:
# align error reads 
!bwa mem data/MG1655-K12.first400K.fasta data/ecoli_400K_err_1.fastq data/ecoli_400K_err_2.fastq >data/ecoli_err_400K.sam

In [2]:
!samtools flagstat data/ecoli_err_400K.sam

2763204 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
2762617 + 0 mapped (99.98% : N/A)
2763204 + 0 paired in sequencing
1381602 + 0 read1
1381602 + 0 read2
2760820 + 0 properly paired (99.91% : N/A)
2762132 + 0 with itself and mate mapped
485 + 0 singletons (0.02% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [ ]:
!spades.py --only-error-correction -1 data/ecoli_400K_err_1.fastq -2 data/ecoli_400K_err_2.fastq -o data/output_spades/

In [ ]:
# align corrected reads 
# !gzip -d data/output_spades/*.gz
!bwa mem data/MG1655-K12.first400K.fasta data/output_spades/corrected/ecoli_400K_err_1.00.0_0.cor.fastq data/output_spades/corrected/ecoli_400K_err_2.00.0_0.cor.fastq >data/ecoli_cor_400K.sam

In [3]:
!samtools flagstat data/ecoli_cor_400K.sam

2716054 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
2715944 + 0 mapped (100.00% : N/A)
2716054 + 0 paired in sequencing
1358027 + 0 read1
1358027 + 0 read2
2714586 + 0 properly paired (99.95% : N/A)
2715876 + 0 with itself and mate mapped
68 + 0 singletons (0.00% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [7]:
def get_genome_seq(path):
    return next(SeqIO.parse(path, "fasta")).seq

def is_skip_processing_reads(error_read, corrected_read):
    return (error_read.cigartuples is None or error_read.query_sequence is None
            or corrected_read.cigartuples is None or corrected_read.query_sequence is None) \
            or error_read.get_aligned_pairs() != corrected_read.get_aligned_pairs()


def get_stats_by_read(error_read, corrected_read, genome):
    if is_skip_processing_reads(error_read, corrected_read):
        return 0, 0, 0, 0, 0, 0
    # aligned_pairs for error and corrected reads are same
    aligned_pairs = error_read.get_aligned_pairs()
    idx_aligned_pairs = 0
    error_query_seq = error_read.query_sequence
    corrected_query_seq = corrected_read.query_sequence

    # main return values
    num_undetected_error = 0
    num_falsely_corrected_error = 0
    num_detected_and_corrected_error = 0
    num_correctly_unmodified_base = 0
    num_detected_and_removed_error = 0
    num_incorectly_removed_base = 0
    #
    size_all_blocks = 0
    for op, length in error_read.cigartuples:
        # skipping hard clipping
        if op == 5:
            continue

        if op == 0 or op == 8:  # match or mismatch
            for i in range(idx_aligned_pairs,
                           idx_aligned_pairs + length):  # walk by read in which match or mismatch with reference
                idx_read = aligned_pairs[i][0]
                idx_reference = aligned_pairs[i][1]
                char_of_error_read = error_query_seq[idx_read].upper()
                char_of_corrected_read = corrected_query_seq[idx_read].upper()
                char_of_reference = genome[idx_reference].upper()

                # update mismatch stat
                if char_of_error_read != char_of_reference: # error in raw data
                    if char_of_corrected_read == char_of_error_read:
                        num_undetected_error += 1
                    if char_of_corrected_read == char_of_reference:
                        if char_of_reference == 'N':
                            num_detected_and_removed_error += 1
                        else:
                            num_detected_and_corrected_error += 1
                else: # correct base in raw data
                    if char_of_corrected_read == char_of_error_read:
                        num_correctly_unmodified_base += 1
                    if char_of_corrected_read != char_of_error_read:
                        if char_of_corrected_read == 'N':
                            num_incorectly_removed_base += 1
                        else:
                            num_falsely_corrected_error += 1

        idx_aligned_pairs += length
        size_all_blocks += length

    return num_undetected_error, num_falsely_corrected_error, num_detected_and_corrected_error, \
           num_correctly_unmodified_base, num_detected_and_removed_error, num_incorectly_removed_base


def get_stats(path_sam_corrected, path_sam_error, genome):
    samfile_error = pysam.AlignmentFile(path_sam_error, "rb")
    samfile_corrected = pysam.AlignmentFile(path_sam_corrected, "rb")

    inner_iterator = samfile_corrected.fetch()
    corrected_read = next(inner_iterator)
    total_num_undetected_error = 0
    total_num_falsely_corrected_error = 0
    total_num_detected_and_corrected_error = 0
    total_num_correctly_unmodified_base = 0
    total_num_detected_and_removed_error = 0
    total_num_incorectly_removed_base = 0

    try:
        for error_read in samfile_error.fetch():
            if error_read.query_name is None:
                raise RuntimeError("These reads have none query_name :(")
            if error_read.query_name != corrected_read.query_name:
                continue
            # error_read.query_name == corrected_read.query_name
            num_ue, num_fc, num_dce, num_cu, num_dr, num_ir = get_stats_by_read(error_read, corrected_read, genome)
            total_num_undetected_error += num_ue
            total_num_falsely_corrected_error += num_fc
            total_num_detected_and_corrected_error += num_dce
            total_num_correctly_unmodified_base += num_cu
            total_num_detected_and_removed_error += num_dr
            total_num_incorectly_removed_base += num_ir
            corrected_read = next(inner_iterator)

    except StopIteration:
        pass

    samfile_error.close()
    samfile_corrected.close()
    return total_num_undetected_error, total_num_falsely_corrected_error, total_num_detected_and_corrected_error, \
           total_num_correctly_unmodified_base, total_num_detected_and_removed_error, total_num_incorectly_removed_base

In [8]:
genome_seq = get_genome_seq("data/MG1655-K12.first400K.fasta")
num_ue, num_fc, num_dce, num_cu, num_dr, num_ir = get_stats("data/ecoli_cor_400K.sam", "data/ecoli_err_400K.sam", genome_seq)
print("Number of undetected errors:", num_ue)
print("Number of falsely corrected errors:", num_fc)
print("Number of detected and corrected errors:", num_dce)
print("Number of correctly unmodified bases:", num_cu)
print("Number of detected and removed_errors:", num_dr)
print("Number of incorectly removed bases:", num_ir)

Number of undetected errors: 136309
Number of falsely corrected errors: 144
Number of detected and corrected errors: 490385
Number of correctly unmodified bases: 178481047
Number of detected and removed_errors: 0
Number of incorectly removed bases: 378482
